# Práctica 6

date: 07/11/2023  
File: Chehade_practica6.ipynb
Author : Pablo Naim Chehade   
Email: pablo.chehade.villalba@gmail.com  
GitHub: https://github.com/Lupama2  

## Ejercicio 1

## Ejercicio 2